<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/3-nmf-naive-bayes-sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NMF y Naïve Bayes

Preparamos el ambiente

In [33]:
!wget https://files.fast.ai/data/examples/imdb_sample.tgz
!tar -xvf imdb_sample.tgz

--2020-11-09 21:52:16--  https://files.fast.ai/data/examples/imdb_sample.tgz
Resolving files.fast.ai (files.fast.ai)... 104.26.3.19, 104.26.2.19, 172.67.69.159, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 571827 (558K) [application/x-gtar-compressed]
Saving to: ‘imdb_sample.tgz.2’

imdb_sample.tgz.2   100%[===================>] 558.42K  2.09MB/s    in 0.3s    

2020-11-09 21:52:16 (2.09 MB/s) - ‘imdb_sample.tgz.2’ saved [571827/571827]

imdb_sample/
imdb_sample/texts.csv


In [34]:
import pandas as pd
import sklearn.datasets
import sklearn.feature_extraction
import sklearn.decomposition
import numpy as np

In [35]:
remove = ('headers', 'footers', 'quotes')
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups = sklearn.datasets.fetch_20newsgroups(remove=remove,categories=categories)

# Actividad 1: NMF

Desde la matriz de tf-idf calculada anteriormente sobre newsgroups realice una descomposición mediante NMF.

1.   Verifique hasta que vector los temas identificados en NMF hacen “sentido”  para los 4 temas seleccionados.
3.   Compare su resultado con SVD.

In [36]:
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(stop_words="english",min_df=0.005)
x = tfidf_vectorizer.fit_transform(newsgroups.data)

In [37]:
vocab = tfidf_vectorizer.get_feature_names()
def show_topics(a):
  num_top_words=8
  top_words_values = lambda t: [f"{vocab[i]}({str(round(v,3))})" for i,v in zip(np.argsort(t)[:-num_top_words-1:-1],np.sort(t)[:-num_top_words-1:-1])]
  topic_words_values = ([top_words_values(t) for t in a])
  return [' '.join(t) for t in topic_words_values]
#NMF
d=5 # num topics
clf = sklearn.decomposition.NMF(n_components=d, random_state=1)
W1 = clf.fit_transform(x)
H1 = clf.components_
show_topics(H1)

['don(0.709) people(0.686) think(0.672) just(0.648) like(0.426) say(0.335) know(0.321) time(0.294)',
 'thanks(0.611) graphics(0.496) files(0.417) image(0.381) file(0.347) program(0.315) know(0.298) format(0.279)',
 'space(1.097) nasa(0.462) launch(0.28) shuttle(0.278) orbit(0.228) lunar(0.222) moon(0.22) earth(0.22)',
 'god(1.44) jesus(0.628) bible(0.413) believe(0.28) christian(0.265) does(0.233) atheism(0.224) christ(0.217)',
 'com(0.501) ico(0.46) tek(0.456) bobbe(0.449) bronx(0.442) beauchaine(0.442) manhattan(0.442) vice(0.44)']

Para este paso práctico utilizaremos el conjunto de datos de imbd en donde tenemos una serie de documentos etiquetados con la polaridad (negativo o positivo) asociada.

In [38]:
imdb = pd.read_csv("imdb_sample/texts.csv")

In [39]:
imdb

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False
...,...,...,...
995,negative,There are many different versions of this one ...,True
996,positive,Once upon a time Hollywood produced live-actio...,True
997,negative,Wenders was great with Million $ Hotel.I don't...,True
998,negative,Although a film with Bruce Willis is always wo...,True


Para construir nuestra matriz de terminos y documentos utilizademos la clase CountVectorizer de sklearn.

In [40]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(stop_words="english")
term_doc_matrix_train = count_vectorizer.fit_transform(imdb[imdb.is_valid == False].text)

Nuestra matriz de términos y documentos tiene la forma (documentos,términos)

In [41]:
term_doc_matrix_train

<800x16163 sparse matrix of type '<class 'numpy.int64'>'
	with 74815 stored elements in Compressed Sparse Row format>

Calculamos la cantidad de apariciones de cada una de las palabras en cada una de las clases, junto con la cantidad de apariciones de cada clase.

In [42]:
c_w_negative = np.asarray(term_doc_matrix_train.todense()[imdb[imdb.is_valid == False].label == "negative",:].sum(0)).reshape(-1) # apariciones de cada una de las palabras en la clase negativa
c_w_positive = np.asarray(term_doc_matrix_train.todense()[imdb[imdb.is_valid == False].label == "positive",:].sum(0)).reshape(-1)
c_negative = (imdb[imdb.is_valid == False].label == "negative").sum() # apariciones de la clase negativa
c_positive = (imdb[imdb.is_valid == False].label == "positive").sum()

## Actividad 2: Palabras más comunes por clase

Dados los arreglos que contienen la cantidad de apariciones de cada palabra en cada clase, imprima las 3 palabras más comunes para cada clase.

In [43]:
vocab = count_vectorizer.get_feature_names()
print(np.array(vocab)[np.argsort(c_w_negative)][-3:])
print(np.array(vocab)[np.argsort(c_w_positive)][-3:])

['film' 'movie' 'br']
['movie' 'film' 'br']


## Actividad 3: Ratio positivo - negativo

Calcule el ratio positivo negativo en esta base de datos, y muestre cuales son las 3 palabras más probables en cada clase.

In [44]:
p_w_negative = (c_w_negative + 1) / (c_negative + len(vocab)) # Probabilidad asociada a cada palabra en la clase negativa
p_w_positive = (c_w_positive + 1) / (c_positive + len(vocab)) 

In [45]:
# Programa
r = np.log(p_w_positive / p_w_negative)
print(np.array(vocab)[np.argsort(r)][:20])
print(np.array(vocab)[np.argsort(r)][-20:])

['porn' 'vargas' 'crap' 'naschy' 'dog' 'crater' 'worst' 'disappointment'
 'soderbergh' 'fuqua' 'scorsese' 'jackman' 'johnson' 'poorly' 'junior'
 'bey' 'rangers' 'dinosaur' 'bet' 'lake']
['chong' 'kitty' 'leon' 'rachel' 'alfred' 'hanzo' 'howard' 'cheech'
 'sport' 'paxton' 'han' 'davies' 'jabba' 'gilliam' 'jimmy' 'felix' 'noir'
 'astaire' 'fanfan' 'biko']


## Actividad 4: Clasificador

Ahora calcule el log ratio de cada documento. Si asumimos que > 0 es opinión positiva, ¿cuál es el accuracy de la clasificación?

In [46]:
# Programa
# term_doc_matrix_train.shape
term_doc_matrix_test = count_vectorizer.transform(imdb[imdb.is_valid == True].text)
test_pred = term_doc_matrix_test.dot(r) > 0
test_label = np.where(imdb[imdb.is_valid == True].label == "positive", True, False)
confusion_matrix = pd.crosstab(test_label,test_pred)
print(np.array(confusion_matrix).diagonal().sum() / np.array(confusion_matrix).sum())

0.67


## Actividad 5: Bigramas

Genere un nuevo clasificador, esta vez agregando bigramas al vocabulario de la matriz de términos y documentos.

In [47]:
# Programa
count_vectorizer_bigrams = sklearn.feature_extraction.text.CountVectorizer(stop_words="english",ngram_range=(1,2))
term_doc_matrix_train_bigrams = count_vectorizer_bigrams.fit_transform(imdb[imdb.is_valid == False].text)
c_w_negative_bigrams = np.asarray(term_doc_matrix_train_bigrams.todense()[imdb[imdb.is_valid == False].label == "negative",:].sum(0)).reshape(-1) # apariciones de cada una de las palabras en la clase negativa
c_w_positive_bigrams = np.asarray(term_doc_matrix_train_bigrams.todense()[imdb[imdb.is_valid == False].label == "positive",:].sum(0)).reshape(-1)
p_w_negative_bigrams = (c_w_negative_bigrams + 1) / c_negative # Probabilidad asociada a cada palabra en la clase negativa
p_w_positive_bigrams = (c_w_positive_bigrams + 1) / c_positive 
r_bigrams = np.log(p_w_positive_bigrams / p_w_negative_bigrams)
term_doc_matrix_test_bigrams = count_vectorizer_bigrams.transform(imdb[imdb.is_valid == True].text)
test_pred_bigrams = term_doc_matrix_test_bigrams.dot(r_bigrams) > 0
confusion_matrix_bigrams = pd.crosstab(test_label,test_pred_bigrams)
print(np.array(confusion_matrix_bigrams).diagonal().sum() / np.array(confusion_matrix_bigrams).sum())

0.77
